Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [138]:
import gc
gc.collect()

0

In [139]:
import functools
from random import random, seed
from itertools import product
import numpy as np
import math

from icecream import ic
#from tqdm.auto import tqdm

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [247]:
UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.3
MAX_STEPS = 1_000

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [248]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)


def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def cost(solution):
    """Returns the cost of a solution (to be minimized) tracking number of calls"""
    return COSTS[solution].sum()

## Helper Functions

In [249]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    coverage = np.logical_or.reduce(SETS[solution])
    is_valid = np.all(coverage)
    return is_valid


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [250]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)
#ic(solution)

(np.True_, np.float64(841073050.1508384))

In [251]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < 1
valid(solution), cost(solution)
#ic(solution)


(np.True_, np.float64(841073050.1508384))

In [252]:
# Start with all sets taken
#initial_solution = np.full(NUM_SETS, True)

solution = rng.random(NUM_SETS) < 1
# ic(initial_solution)

## Helpers


In [253]:
def tweak(solution):
    new_solution = np.copy(solution)
    pos = rng.integers(low=0, high=NUM_SETS)
    new_solution[pos] = not solution[pos]
    return new_solution

def multiple_mutation(solution: np.ndarray) -> np.ndarray:
    new_sol = solution.copy()
    mask = rng.random(NUM_SETS) < 0.05
    new_sol = np.logical_xor(solution, mask)
    return new_sol

def fitness(solution):
    return (valid(solution), -cost(solution))

## Random Mutation Hill Climber

In [254]:
solution = rng.random(NUM_SETS) < 1
solution_fitness = fitness(solution)
ic(fitness(solution))
if (UNIVERSE_SIZE < 1000 and NUM_SETS < 100):
    steps = MAX_STEPS*1000
    ic(steps)
else:
    steps = MAX_STEPS
    ic(steps)
for _ in range(steps):
    new_solution = tweak(solution)
    new_fitness = fitness(new_solution)
    if new_fitness > solution_fitness:
        solution = new_solution

fitness(solution)
# ic(current_solution)

ic| fitness(solution): (np.True_, np.float64(-841073050.1508384))
ic| steps: 1000


(np.True_, np.float64(-765056846.9102377))

## Simulated Annealing

In [255]:
INITIAL_TEMPERATURE = 1000  
COOLING_RATE = 0.995 
MIN_TEMPERATURE = 1e-4 


In [256]:

solution = rng.random(NUM_SETS) < 1
solution_fitness = fitness(solution)
ic(solution_fitness)
#ic(fitness(solution))
temperature = INITIAL_TEMPERATURE

for _ in range(min(steps,MAX_STEPS)):
    #if temperature <= MIN_TEMPERATURE:
    #    break

    new_solution = multiple_mutation(solution)
    new_fitness = fitness(new_solution)
    #ic(new_fitness)

    if valid(new_solution) & (new_fitness[1] > solution_fitness[1] or np.exp((new_fitness[1]-solution_fitness[1]) / temperature) > rng.random()):
        solution = new_solution
        solution_fitness = new_fitness

    temperature *= COOLING_RATE

ic(fitness(solution))
None

ic| solution_fitness: (np.True_, np.float64(-841073050.1508384))
ic| fitness(solution): (np.True_, np.float64(-379306610.56588715))
